In [1]:
from gru_torch import (
    GRU,
    SequenceDataset,
    create_sliding_window,
    arithmetic_progression,
    mape,
    DataLoader,
    torch
)
import plotly.graph_objects as go
import numpy as np

In [8]:
def scale_sequence(sequence):
    mean = np.mean(sequence)
    deviation = np.std(sequence)    
    scaled = (np.array(sequence) - mean) / deviation    
    return scaled, mean, deviation

def descale_sequence(scaled, mean, deviation):
    descaled = scaled * deviation + mean
    return descaled

def predict_next_n_elements(sequence, window_size, batch_size, hidden_size, n,
                            max_epochs, verbosity, lr, use_adam: bool = False):
    scaled, mean, deviation = scale_sequence(sequence[:-n])        
    # scaled = sequence[:-n]  
    X, y = create_sliding_window(scaled, window_size, n)
    model = GRU(window_size, hidden_size, n, num_layers=1,
                dropout=0, learning_rate=lr, num_epochs=max_epochs)
    train_loader = DataLoader(dataset=SequenceDataset(X, y),
                              batch_size=batch_size,
                              shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train_model(train_loader, device, verbosity, use_adam)
    train_pred = model.predict(train_loader, device).numpy()
    training_mape = mape(y, train_pred)           
    x_last = np.expand_dims(scaled[-window_size:], axis=0)
    train_loader = DataLoader(dataset=SequenceDataset(x_last, np.expand_dims(sequence[-n:], axis=0)),
                              batch_size=batch_size,
                              shuffle=False)     
    pred = model.predict(train_loader, device).numpy()
    descaled_pred = descale_sequence(pred[-1], mean, deviation)
    # descaled_pred = pred[-1]
    return training_mape, model, descaled_pred, mean, deviation

In [10]:
experiments_amount = 4
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 8
hidden_size = 6
output_size = 3

verbosity = 100000
lr = 1e-2
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):        
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)
            

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!


In [11]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()